# Working with Sea Level Rise data in Cal-Adapt API

In [1]:
import requests 
import json
headers = {'ContentType': 'json'}

api = 'http://api.cal-adapt.org/api'
resource = 'rstores'


### Get a list of sea level rise datasets from Cal-Adapt API

In [2]:
search_str = 'Sea+Level+Rise'
params = {'name': search_str, 'pagesize': 20}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
url = api + '/' + resource + '/' + '?' + params_str
url

'http://api.cal-adapt.org/api/rstores/?name=Sea+Level+Rise&pagesize=20'

In [3]:
response = requests.get(url, headers=headers)
data = response.json()
results = data['results']
for item in results:
    print(item['url'], item['name'])


http://api.cal-adapt.org/api/rstores/slr_coast_0_0m_jdr/ Coast Mosaic Sea Level Rise 0.0m
http://api.cal-adapt.org/api/rstores/slr_coast_0_5m_jdr/ Coast Mosaic Sea Level Rise 0.5m
http://api.cal-adapt.org/api/rstores/slr_coast_1_0m_jdr/ Coast Mosaic Sea Level Rise 1.0m
http://api.cal-adapt.org/api/rstores/slr_coast_1_41m_jdr/ Coast Mosaic Sea Level Rise 1.41m
http://api.cal-adapt.org/api/rstores/slr_delta_0_0m_jdr/ Delta Sea Level Rise 0.0m
http://api.cal-adapt.org/api/rstores/slr_delta_0_5m_jdr/ Delta Sea Level Rise 0.5m
http://api.cal-adapt.org/api/rstores/slr_delta_1_0m_jdr/ Delta Sea Level Rise 1.0m
http://api.cal-adapt.org/api/rstores/slr_delta_1_41m_jdr/ Delta Sea Level Rise 1.41m
http://api.cal-adapt.org/api/rstores/slr_sfbay_0_0m_jdr/ San Francisco Bay Sea Level Rise 0.0m
http://api.cal-adapt.org/api/rstores/slr_sfbay_0_5m_jdr/ San Francisco Bay Sea Level Rise 0.5m
http://api.cal-adapt.org/api/rstores/slr_sfbay_1_0m_jdr/ San Francisco Bay Sea Level Rise 1.0m
http://api.cal-adap

### Generate parameter for your location

In [4]:
lat = 38.106914
lng = -121.456679
point = '{"type":"Point","coordinates":[' + str(lng) + ',' + str(lat) + ']}' #geojson format
point = 'POINT(' + str(lng) + ' ' + str(lat) + ')' #WKT format

### Query SLR datasets for your location

In [5]:
params = {'g': point}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
print('Depth of flooding depth (meters) for point location:', lat, lng, sep=' ')
print()
for item in results:
    url = item['url'] + '?' + params_str
    response = requests.get(url, headers=headers)
    # Request will return a server error if point is outside spatial extent of SLR dataset
    if response.status_code == requests.codes.ok:
        data = response.json()
        if float(data['image']) == float(data['nodata']):
            print(item['name'], 'No Data', sep=' - ')
        else:
            print(item['name'], data['image'], sep=' - ')
    else:
        print(item['name'], 'Outside extent', sep=' - ')

Depth of flooding depth (meters) for point location: 38.106914 -121.456679

Coast Mosaic Sea Level Rise 0.0m - No Data
Coast Mosaic Sea Level Rise 0.5m - No Data
Coast Mosaic Sea Level Rise 1.0m - No Data
Coast Mosaic Sea Level Rise 1.41m - No Data
Delta Sea Level Rise 0.0m - 0.8650000095367432
Delta Sea Level Rise 0.5m - 0.949999988079071
Delta Sea Level Rise 1.0m - 6.008999824523926
Delta Sea Level Rise 1.41m - 7.077000141143799
San Francisco Bay Sea Level Rise 0.0m - Outside extent
San Francisco Bay Sea Level Rise 0.5m - Outside extent
San Francisco Bay Sea Level Rise 1.0m - Outside extent
San Francisco Bay Sea Level Rise 1.41m - Outside extent


### Notes:
* The `image` field contains the value for Sea Level Rise. If the location does not have any Sea Level Rise, the `image` field will have the same value as the `nodata` field
* Cal-Adapt provides inundation depth mosaics derived from the original source layers for 3 areas:
    * San Francisco Bay Area (3 meter resolution)
    * Sacramento-SanJoaquin Delta (3 meter resolution)
    * California coast (50 meter resolution)
* For 4 different SLR scenarios
    * 0 meters
    * 0.5 meters
    * 1 meter
    * 1.41 meters
* There are overlaps between spatial extents of the 3 areas. So querying all the SLR datasets for a point location may return results for multiple areas. If a point is not contained within spatial extent of a SLR dataset, you will get a server (500) error.